# Import Packages

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# Read Data

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
PATH = '/content/drive/MyDrive/Citadel Datathon/Data/Ziao/'

df_match = pd.read_csv(PATH+'match.csv')
df_team_attr = pd.read_csv(PATH+'team_attributes.csv')
df_match.date = pd.to_datetime(df_match.date)
df_team_attr.date = pd.to_datetime(df_team_attr.date)

In [ ]:
df_player_attr = pd.read_csv(PATH+'player_data.csv')

# Add Team into Match

In [ ]:
# Drop country id as league id is its duplication
# Drop bidding information
# Drop Goal information
df_match = df_match[[i for i in df_match.columns if i[0].islower()]]
del df_match['country_id']
del df_match['home_team_goal']
del df_match['away_team_goal']

In [ ]:
df_team_attr_home = df_team_attr.copy()
df_team_attr_away = df_team_attr.copy()

In [ ]:
df_team_attr_home.columns = ['home_'+i for i in df_team_attr.columns]
df_team_attr_away.columns = ['away_'+i for i in df_team_attr.columns]

In [ ]:
df_match = pd.merge(df_match, df_team_attr_home, on=['home_team_id'])
df_match['home_date_delta'] = df_match.date - df_match.home_date
df_match['home_date_delta_days'] = df_match['home_date_delta'].dt.days
df_match = df_match[df_match.home_date_delta_days>0]
df_match = df_match.loc[df_match.groupby(['match_id']).home_date_delta_days.idxmin()].reset_index(drop=True)
del df_match['home_date_delta']
del df_match['home_date_delta_days']

In [ ]:
df_match = pd.merge(df_match, df_team_attr_away, on=['away_team_id'])
df_match['away_date_delta'] = df_match.date - df_match.away_date
df_match['away_date_delta_days'] = df_match['away_date_delta'].dt.days
df_match = df_match[df_match.away_date_delta_days>0]
df_match = df_match.loc[df_match.groupby(['match_id']).away_date_delta_days.idxmin()].reset_index(drop=True)
del df_match['away_date_delta']
del df_match['away_date_delta_days']

In [ ]:
df_match

,league_id,season,stage,date,match_id,home_team_id,away_team_id,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,home_date,home_buildUpPlaySpeed,home_buildUpPlayPassing,home_chanceCreationPassing,home_chanceCreationCrossing,home_chanceCreationShooting,home_defencePressure,home_defenceAggression,home_defenceTeamWidth,away_date,away_buildUpPlaySpeed,away_buildUpPlayPassing,away_chanceCreationPassing,away_chanceCreationCrossing,away_chanceCreationShooting,away_defencePressure,away_defenceAggression,away_defenceTeamWidth
0,4769,2009/2010,14,2010-04-07,654153,8592,9874,31293,25541,30670,38703,26052,26311,40964,26163,78185,40060,40197,94324,26328,147351,36123,42304,26345,155623,26227,147958,134222,41238,2010-02-22,55,45,70,65,70,65,70,65,2010-02-22,66,70,35,40,50,65,30,65
1,4769,2009/2010,25,2010-03-10,654399,9827,8583,26168,40543,26119,41092,39978,33744,26144,154230,179393,40710,25537,26241,11323,30823,26136,39996,39993,26199,115429,26288,39994,34037,2010-02-22,70,35,70,50,70,55,70,70,2010-02-22,30,70,35,65,35,30,55,30
2,4769,2009/2010,26,2010-04-14,654423,8682,9827,107806,40680,108535,94013,165583,40687,111787,26057,37466,40683,179403,26168,26142,40543,26119,41092,26144,39963,30907,179393,40710,41119,2010-02-22,70,45,65,60,65,70,60,70,2010-02-22,70,35,70,50,70,55,70,70
3,4769,2009/2010,26,2010-02-28,654425,8639,8583,30742,30914,26107,77741,26291,26160,26195,56916,32575,107417,35407,26241,11323,30823,26136,39996,26261,39993,26199,115429,26288,39994,2010-02-22,70,60,50,45,55,55,70,55,2010-02-22,30,70,35,65,35,30,55,30
4,4769,2009/2010,26,2010-02-27,654427,8689,9874,40709,23075,46539,46540,33122,26324,41232,26121,26364,26132,150296,26359,157860,25519,147351,36123,41348,42304,155623,147958,115160,41238,2010-02-22,45,70,30,50,40,70,60,70,2010-02-22,66,70,35,40,50,65,30,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16909,19694,2015/2016,38,2016-05-14,2216668,8597,9938,32584,744907,302372,159663,171789,246316,419136,430433,0,80293,159373,620136,654084,190846,417756,14039,640641,540680,34447,0,27974,244035,2015-09-10,50,60,49,62,47,46,46,52,2015-09-10,60,53,55,49,45,48,42,53
16910,19694,2015/2016,38,2016-05-14,2216669,8426,8429,160367,39079,149174,32088,0,308936,582500,212715,425251,195678,163215,32845,127140,244953,241424,132111,430429,282295,190493,632488,642517,440404,2015-09-10,48,47,47,53,60,46,46,49,2015-09-10,48,47,73,53,71,55,48,52
16911,19694,2015/2016,38,2016-05-15,2216670,8485,8649,528212,98057,171094,551805,32419,25416,35463,32455,119435,190851,553798,109622,21914,32833,35955,135335,262753,210382,369742,35398,95793,24439,2015-09-10,59,53,51,72,63,49,45,63,2015-09-10,45,60,46,49,58,36,53,34
16912,19694,2015/2016,38,2016-05-15,2216671,9925,9927,32562,582499,166471,144989,31701,111060,70078,527549,303899,237563,439369,237562,306651,41060,696404,609711,34563,523758,35451,41059,32826,196865,2015-09-10,54,53,51,51,62,58,55,56,2015-09-10,64,55,44,45,27,37,44,51


In [ ]:
home_cols = [i for i in df_team_attr_home.columns if 'home' in i]

In [ ]:
for col in home_cols:
  if col in df_match.columns:
    df_match[col.replace('home_',"")+'Diff'] = df_match[col] - df_match[col.replace('home','away')]

In [ ]:
for col in home_cols:
  if col in df_match.columns:
    del df_match[col]
    del df_match[col.replace('home','away')]

# Add Player into Match

In [ ]:
# Drop the index column
df_player_attr = df_player_attr[df_player_attr.columns[1:]]

In [ ]:
df_player_attr.columns

Index(['player_id', 'player_name', 'birthday', 'height', 'weight', 'date',
       'overall_rating', 'potential', 'preferred_foot', 'attacking_work_rate',
       'defensive_work_rate', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes', 'yr', 'month', 'age', 'season'],
      dtype='object')

In [ ]:
# Drop columns not needed
df_player_attr = df_player_attr[[i for i in df_player_attr.columns if i not in ['player_name', 'birthday','yr','month','season']]]

In [ ]:
df_player_attr.date = pd.to_datetime(df_player_attr.date)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
# Getting the player info from match
df_match_player = df_match[['match_id','date'] + [i for i in df_match.columns if 'player' in i]]

In [ ]:
df_match_player = df_match_player.melt(id_vars=["match_id",'date'], 
        var_name='player', 
        value_name="player_id")

In [ ]:
df_player_attr.rename(columns={'date':'player_date'}, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
df_match_player = pd.merge(df_match_player, df_player_attr, on=['player_id'])

In [ ]:
df_match_player['date_delta'] = df_match_player.date - df_match_player.player_date
df_match_player['date_delta_days'] = df_match_player['date_delta'].dt.days
df_match_player = df_match_player[df_match_player.date_delta_days>0]
df_match_player = df_match_player.loc[df_match_player.groupby(['match_id','player_id']).date_delta_days.idxmin()].reset_index(drop=True)
del df_match_player['date_delta']
del df_match_player['date_delta_days']

In [ ]:
import gc
gc.collect()

140

In [ ]:
df_match_player['home_or_away'] = df_match_player.player.apply(lambda x: x.split("_")[0])

In [ ]:
df_match_player.dtypes

match_id                        int64
date                   datetime64[ns]
player                         object
player_id                       int64
height                        float64
weight                          int64
player_date            datetime64[ns]
overall_rating                float64
potential                     float64
preferred_foot                 object
attacking_work_rate            object
defensive_work_rate            object
crossing                      float64
finishing                     float64
heading_accuracy              float64
short_passing                 float64
volleys                       float64
dribbling                     float64
curve                         float64
free_kick_accuracy            float64
long_passing                  float64
ball_control                  float64
acceleration                  float64
sprint_speed                  float64
agility                       float64
reactions                     float64
balance     

In [ ]:
cat_cols = ['preferred_foot','attacking_work_rate','defensive_work_rate']
id_cols = ['match_id','home_or_away','date','player','player_id','player_date']

# Get Max
max_df = df_match_player.groupby(['match_id','home_or_away'],as_index=False)[[i for i in df_match_player.columns if i not in cat_cols + id_cols]].max()
max_home_df = max_df[max_df.home_or_away == 'home']
max_away_df = max_df[max_df.home_or_away == 'away']
del max_home_df['home_or_away']
del max_away_df['home_or_away']
max_home_df.columns = ['home_max_'+i for i in max_home_df.columns]
max_away_df.columns = ['away_max_'+i for i in max_away_df.columns]
max_df = pd.merge(max_home_df, max_away_df, left_on='home_max_match_id', right_on='away_max_match_id')
for col in max_home_df.columns[1:]:
  max_df[col.replace('home_',"")+'_diff'] = max_df[col] - max_df[col.replace('home','away')]
for col in max_home_df.columns[1:]:
  del max_df[col]
  del max_df[col.replace('home','away')]

In [ ]:
# Get Average
avg_df = df_match_player.groupby(['match_id','home_or_away'],as_index=False)[[i for i in df_match_player.columns if i not in cat_cols + id_cols]].mean()
avg_home_df = avg_df[avg_df.home_or_away == 'home']
avg_away_df = avg_df[avg_df.home_or_away == 'away']
del avg_home_df['home_or_away']
del avg_away_df['home_or_away']
avg_home_df.columns = ['home_avg_'+i for i in avg_home_df.columns]
avg_away_df.columns = ['away_avg_'+i for i in avg_away_df.columns]
avg_df = pd.merge(avg_home_df, avg_away_df, left_on='home_avg_match_id', right_on='away_avg_match_id')
for col in avg_home_df.columns[1:]:
  avg_df[col.replace('home_',"")+'_diff'] = avg_df[col] - avg_df[col.replace('home','away')]
for col in avg_home_df.columns[1:]:
  del avg_df[col]
  del avg_df[col.replace('home','away')]

In [ ]:
# Get Std
std_df = df_match_player.groupby(['match_id','home_or_away'],as_index=False)[[i for i in df_match_player.columns if i not in cat_cols + id_cols]].std()
std_home_df = std_df[std_df.home_or_away == 'home']
std_away_df = std_df[std_df.home_or_away == 'away']
del std_home_df['home_or_away']
del std_away_df['home_or_away']
std_home_df.columns = ['home_std_'+i for i in std_home_df.columns]
std_away_df.columns = ['away_std_'+i for i in std_away_df.columns]
std_df = pd.merge(std_home_df, std_away_df, left_on='home_std_match_id', right_on='away_std_match_id')
for col in std_home_df.columns[1:]:
  std_df[col.replace('home_',"")+'_diff'] = std_df[col] - std_df[col.replace('home','away')]
for col in std_home_df.columns[1:]:
  del std_df[col]
  del std_df[col.replace('home','away')]

In [ ]:
cat_df = df_match_player.groupby(['match_id','home_or_away'],as_index=False)[[i for i in df_match_player.columns if i in cat_cols]].apply(lambda x: x.mode().iloc[0])

In [ ]:
cat_home_df = cat_df[cat_df.home_or_away == 'home']
cat_away_df = cat_df[cat_df.home_or_away == 'away']
del cat_home_df['home_or_away']
del cat_away_df['home_or_away']
cat_home_df.columns = ['home_mode_'+i for i in cat_home_df.columns]
cat_away_df.columns = ['away_mode_'+i for i in cat_away_df.columns]
cat_df = pd.merge(cat_home_df, cat_away_df, left_on='home_mode_match_id', right_on='away_mode_match_id')

In [ ]:
df_final = df_match.copy()
df_final = pd.merge(df_final, max_df, left_on='match_id', right_on='home_max_match_id')
df_final = pd.merge(df_final, avg_df, left_on='match_id', right_on='home_avg_match_id')
df_final = pd.merge(df_final, std_df, left_on='match_id', right_on='home_std_match_id')
df_final = pd.merge(df_final, cat_df, left_on='match_id', right_on='home_mode_match_id')

In [ ]:
df_final = df_final[['match_id','league_id'] + [i for i in df_final if 'id' not in i and 'player' not in i and 'dateDiff' not in i]]

In [ ]:
df_final

,match_id,league_id,season,stage,date,buildUpPlaySpeedDiff,buildUpPlayPassingDiff,chanceCreationPassingDiff,chanceCreationCrossingDiff,chanceCreationShootingDiff,defencePressureDiff,defenceAggressionDiff,max_height_diff,max_weight_diff,max_overall_rating_diff,max_potential_diff,max_crossing_diff,max_finishing_diff,max_heading_accuracy_diff,max_short_passing_diff,max_volleys_diff,max_dribbling_diff,max_curve_diff,max_free_kick_accuracy_diff,max_long_passing_diff,max_ball_control_diff,max_acceleration_diff,max_sprint_speed_diff,max_agility_diff,max_reactions_diff,max_balance_diff,max_shot_power_diff,max_jumping_diff,max_stamina_diff,max_strength_diff,max_long_shots_diff,max_aggression_diff,max_interceptions_diff,max_positioning_diff,max_vision_diff,...,std_potential_diff,std_crossing_diff,std_finishing_diff,std_heading_accuracy_diff,std_short_passing_diff,std_volleys_diff,std_dribbling_diff,std_curve_diff,std_free_kick_accuracy_diff,std_long_passing_diff,std_ball_control_diff,std_acceleration_diff,std_sprint_speed_diff,std_agility_diff,std_reactions_diff,std_balance_diff,std_shot_power_diff,std_jumping_diff,std_stamina_diff,std_strength_diff,std_long_shots_diff,std_aggression_diff,std_interceptions_diff,std_positioning_diff,std_vision_diff,std_penalties_diff,std_marking_diff,std_standing_tackle_diff,std_gk_diving_diff,std_gk_handling_diff,std_gk_kicking_diff,std_gk_positioning_diff,std_gk_reflexes_diff,std_age_diff,home_mode_preferred_foot,home_mode_attacking_work_rate,home_mode_defensive_work_rate,away_mode_preferred_foot,away_mode_attacking_work_rate,away_mode_defensive_work_rate
0,654153,4769,2009/2010,14,2010-04-07,-11,-25,35,25,20,0,40,0.00,15,11.0,2.0,5.0,17.0,13.0,14.0,17.0,2.0,5.0,6.0,18.0,11.0,8.0,9.0,1.0,10.0,10.0,17.0,15.0,3.0,15.0,9.0,5.0,10.0,13.0,7.0,...,-2.651720,1.936448,7.996999,1.909638,-2.788291,6.268377,5.676309,2.147803,0.236433,1.113798,1.365838,0.952569,-3.515583,4.224007,-3.361490,0.605078,2.389623,-3.889881,-5.857272,1.246822,6.389257,-6.335066,-7.248492,-8.237951,-10.450849,-2.476971,3.920191,3.377566,7.107857,6.641322,11.358563,8.145229,6.833637,0.285980,right,medium,medium,right,medium,medium
1,654399,4769,2009/2010,25,2010-03-10,40,-35,35,-15,35,25,15,5.08,7,6.0,7.0,10.0,5.0,8.0,3.0,-3.0,14.0,11.0,10.0,-3.0,6.0,-2.0,5.0,10.0,5.0,0.0,0.0,3.0,11.0,3.0,13.0,7.0,11.0,4.0,6.0,...,-1.949116,2.914956,1.108839,0.679470,1.691608,2.967884,4.288011,5.926811,7.534440,-9.067970,2.324250,-3.079042,1.669764,1.650057,-2.456727,-1.332337,-1.656289,0.757876,5.662980,0.755884,5.193807,-3.837149,-2.845207,-7.995925,-4.251284,0.111899,-4.170123,1.991075,4.418856,0.700074,-9.067970,1.901541,7.007729,0.174641,right,medium,medium,right,medium,medium
2,654423,4769,2009/2010,26,2010-04-14,0,10,-5,10,-5,15,-10,2.54,-2,-8.0,-3.0,-11.0,-11.0,-8.0,-8.0,-4.0,-7.0,-19.0,-10.0,-5.0,-9.0,-8.0,-9.0,-3.0,-8.0,3.0,-6.0,-5.0,1.0,0.0,-12.0,-7.0,-16.0,-16.0,-9.0,...,1.365801,-5.713124,-4.699075,-1.975884,-0.145584,-4.607918,-6.377283,-6.787118,-9.077813,5.852567,-3.999284,0.028312,-4.629113,-2.927084,3.103484,2.342771,-1.254352,-3.040510,-5.032008,6.977809,-4.678297,-4.504302,5.353761,3.086993,4.228784,1.046891,-1.265580,-0.673364,-6.946718,-4.263658,11.930710,-4.402520,-6.180520,1.545915,right,medium,medium,right,medium,medium
3,654425,4769,2009/2010,26,2010-02-28,40,-10,15,-20,20,25,15,2.54,9,2.0,6.0,3.0,0.0,-10.0,2.0,12.0,12.0,6.0,10.0,8.0,8.0,1.0,9.0,5.0,9.0,0.0,0.0,-6.0,2.0,-3.0,2.0,-1.0,10.0,-1.0,2.0,...,0.064702,-1.258875,-0.552099,-1.552111,1.781649,6.085962,3.071006,1.486619,6.899813,-1.827869,0.780466,-4.514272,-2.099516,-1.152109,-1.754723,-0.876881,1.314839,2.003180,-0.251649,1.089020,5.243060,-5.717018,-0.902041,-0.228950,-0.354961,-0.429283,1.550895,0.404419,2.800040,1.642829,-1.827869,1.643207,5.550965,0.623641,right,medium,medium,right,medium,medium
4,654427,4769,2009/2010,26,2010-02-27,-21,0,-5,10,-10,5,30,0.00,17,1.0,-4.0,-1.0,-3.0,6.0,1.0,4.0,-8.0,-4.0,-1.0,-1.0,-1.0,3.0,3.0,-1.0,0.0,-4.0,3.0,11.0,3.0,-3.0,2.0,-6.0,16.0,

In [ ]:
df_final.shape

(16914, 129)

In [ ]:
df_final.to_csv(PATH + 'data_raw.csv', index=False)